In [ ]:
!pip install prophet

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
import os
from sklearn.metrics import mean_squared_log_error
import datetime
import math
from prophet import Prophet

In [ ]:
dir_path = '/kaggle/input/store-sales-time-series-forecasting/'

holidays = pd.read_csv(dir_path + 'holidays_events.csv')
oil = pd.read_csv(dir_path + 'oil.csv')
stores = pd.read_csv(dir_path + 'stores.csv')
test = pd.read_csv(dir_path + 'test.csv')
train = pd.read_csv(dir_path + 'train.csv')
transactions = pd.read_csv(dir_path + 'transactions.csv')

In [ ]:
holidays.head()

In [ ]:
holidays.info()

In [ ]:
holidays.describe()

In [ ]:
holidays['type'].unique()

In [ ]:
holidays['locale'].unique()

In [ ]:
holidays['locale_name'].unique()

In [ ]:
holidays.loc[holidays['transferred']==True]

In [ ]:
holidays.groupby(['date']).count()

In [ ]:
holidays.loc[holidays['date']=='2017-12-22']

In [ ]:
holidays[holidays['date'].duplicated()==True]

In [ ]:
oil.head()

In [ ]:
oil.info()

In [ ]:
oil.describe()

In [ ]:
oil['dcoilwtico'] = oil['dcoilwtico'].fillna(method='bfill')
oil.dcoilwtico.isnull().sum()

In [ ]:
oil.head()

In [ ]:
oil.info()

In [ ]:
stores.head()

In [ ]:
stores.describe()

In [ ]:
train.head()

In [ ]:
train.info()

In [ ]:
train.isna().sum()

In [ ]:
train.describe()

In [ ]:
test.head()

In [ ]:
test.info()

In [ ]:
test.describe()

In [ ]:
transactions.head()

In [ ]:
df_data = train
df_data = df_data.merge(stores, how="left", on='store_nbr')   
df_data = df_data.merge(oil, how="left", on='date')      
df_data = df_data.merge(transactions, how="left", on=['date','store_nbr'])  
df_data = df_data.merge(holidays,on='date',how='left')
df_data = df_data.rename(columns={'type_x' : 'store_type','type_y':'holiday_type'})

df_data.date = pd.to_datetime(df_data.date)
df_data['year'] = df_data['date'].dt.year
df_data['month'] = df_data['date'].dt.month
df_data['week'] = df_data['date'].dt.isocalendar().week
df_data['quarter'] = df_data['date'].dt.quarter
df_data['day_of_week'] = df_data['date'].dt.day_name()
df_data.head()

In [ ]:
df_agg = df_data.groupby(['date','store_nbr','family'])['sales'].sum()
df_agg

In [ ]:
col_sales = df_data.groupby('date').sum('sales').reset_index()
sns.relplot(x = col_sales.date, y =  col_sales.sales, kind = 'line', hue = col_sales.date.dt.year)
plt.show()

In [ ]:
# train_x = df_data[df_data['date']<= datetime.datetime(2017,8,15)][['date','sales']].groupby('date').mean().reset_index('date')
# train_x.columns = ['ds', 'y']
# test_x = pd.DataFrame(df_data[df_data['date']>= datetime.datetime(2017,8,16)]['date'])
# test_x.columns = ['ds']
# test_y = pd.DataFrame(df_data[df_data['date']>= datetime.datetime(2017,8,16)]['sales'])
# test_y.columns = ['y']

In [ ]:
# Train the model
# model = Prophet()
# model.fit(train_x)
# y_pred = model.predict(test_x)

In [ ]:
# mean_squared_log_error(test_y, y_pred['yhat'])

In [ ]:
train_data = df_data[['date','sales']].groupby('date').mean().reset_index('date')
train_data.columns = ['ds', 'y']
test_data = test[['date']]
test_data.columns = ['ds']

In [ ]:
# Train the model
model = Prophet()
model.fit(train_data)
y_pred = model.predict(test_data)

In [ ]:
y_pred.head()

In [ ]:
y_pred = y_pred[['ds','yhat']]
y_pred.columns = ['id','sales']

In [ ]:
y_pred['id'] = y_pred['id'].astype('datetime64[ns]').astype(np.int32)

In [ ]:
y_pred=y_pred.drop_duplicates(keep='first')

In [ ]:
y_pred.to_csv('submission.csv')

In [ ]:
f, ax = plt.subplots(1)
f.set_figheight(10)
f.set_figwidth(22)

model.plot(y_pred, ax=ax)
sns.lineplot(x=train_data['ds'], y=train_data['y'], ax=ax, color='orange') #navajowhite

#ax.set_title(f'Prediction \n MAE: {score_mae:.2f}, RMSE: {score_rmse:.2f}', fontsize=14)
ax.set_xlabel(xlabel='Date', fontsize=14)
ax.set_ylabel(ylabel='Sales', fontsize=14)
ax.set_title('Average Sales per Day')

plt.show()

In [ ]:
# train_multivariate = df_data[df_data['date']<= datetime.datetime(2017,8,15)][['date','sales','dcoilwtico']].groupby('date').mean().reset_index('date')
# train_multivariate.columns = ['ds', 'y','dcoilwtico']
# x_valid = pd.DataFrame(df_data[df_data['date']>= datetime.datetime(2017,8,16)][['date','dcoilwtico']])
# x_valid.columns = ['ds','dcoilwtico']

# # Train the model
# model_multivariate = Prophet()
# model_multivariate.add_regressor('dcoilwtico')

# model_multivariate.fit(train_multivariate)
# y_pred_multivariate = model_multivariate.predict(x_valid)